## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import the API key.
import os
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("g_maps_key")

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Tasiilaq,GL,65.6145,-37.6368,41.16,87,93,3.51,overcast clouds
1,1,Vila Franca Do Campo,PT,37.7167,-25.4333,74.19,88,40,2.98,scattered clouds
2,2,Belmonte,BR,-15.8631,-38.8828,70.25,85,56,6.35,broken clouds
3,3,Nogliki,RU,51.8333,143.1667,65.88,72,58,11.34,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,45.48,86,80,4.36,broken clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
5,5,Hithadhoo,MV,-0.6000,73.0833,81.90,72,75,7.99,broken clouds
12,12,Kavieng,PG,-2.5744,150.7967,84.47,71,8,8.97,light rain
15,15,Jamestown,US,42.0970,-79.2353,76.01,89,40,9.22,scattered clouds
18,18,Dingle,PH,10.9995,122.6711,86.65,71,89,1.12,overcast clouds
23,23,Georgetown,MY,5.4112,100.3354,84.88,81,20,1.01,light rain
32,32,Kudahuvadhoo,MV,2.6708,72.8944,81.99,73,100,11.07,overcast clouds
36,36,Rikitea,PF,-23.1203,-134.9692,75.42,80,17,15.95,few clouds
39,39,Salalah,OM,17.0151,54.0924,75.29,100,100,3.44,light intensity drizzle
41,41,Padang,ID,-0.9492,100.3543,75.25,93,100,1.81,light rain
43,43,Princeton,US,25.5384,-80.4089,85.01,82,40,11.50,scattered clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID            0
City               0
Country            0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [22]:
preferred_cities_df.count()

City_ID            225
City               225
Country            225
Lat                225
Lng                225
Max Temp           225
Humidity           225
Cloudiness         225
Wind Speed         225
Current Weather    225
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
5,Hithadhoo,MV,81.90,broken clouds,-0.6000,73.0833,
12,Kavieng,PG,84.47,light rain,-2.5744,150.7967,
15,Jamestown,US,76.01,scattered clouds,42.0970,-79.2353,
18,Dingle,PH,86.65,overcast clouds,10.9995,122.6711,
23,Georgetown,MY,84.88,light rain,5.4112,100.3354,
32,Kudahuvadhoo,MV,81.99,overcast clouds,2.6708,72.8944,
36,Rikitea,PF,75.42,few clouds,-23.1203,-134.9692,
39,Salalah,OM,75.29,light intensity drizzle,17.0151,54.0924,
41,Padang,ID,75.25,light rain,-0.9492,100.3543,
43,Princeton,US,85.01,scattered clouds,25.5384,-80.4089,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    

    # 6c. Get latitude and longitude from DataFrame

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(20)       


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
5,Hithadhoo,MV,81.90,broken clouds,-0.6000,73.0833,Scoop Guest House
12,Kavieng,PG,84.47,light rain,-2.5744,150.7967,Nusa Island Retreat
15,Jamestown,US,76.01,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Dingle,PH,86.65,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
23,Georgetown,MY,84.88,light rain,5.4112,100.3354,Cititel Penang
32,Kudahuvadhoo,MV,81.99,overcast clouds,2.6708,72.8944,Niyama Private Islands Maldives
36,Rikitea,PF,75.42,few clouds,-23.1203,-134.9692,People ThankYou
39,Salalah,OM,75.29,light intensity drizzle,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
41,Padang,ID,75.25,light rain,-0.9492,100.3543,Mercure Padang
43,Princeton,US,85.01,scattered clouds,25.5384,-80.4089,American Best Inn


In [27]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}
<dt>Current Weather<dd>{Current Weather} and {Max Temp} °F
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure

fig



Figure(layout=FigureLayout(height='420px'))